In [ ]:
from numpy import sqrt
import pandas as pd

from risk_data import get_factor_data, calculate_returns_set, get_factor_master
from risk_chart import draw_volatility, draw_correlation, draw_beta, draw_distance_from_ma
from risk_config import HALFLIFES


In [ ]:
# halflifes = [21, 63, 126, 252, 512]
data = get_factor_data()
# factor_master = pd.DataFrame(data['factor_name'].attrs).T
# factor_list = factor_master.index.to_list()

factor_name = 'QQQ'
factor_name_1 = 'IWM'
data

In [ ]:
factor_name = 'QQQ'
factor_name_1 = 'IWM'

In [ ]:
from risk_chart import px_line
da = data.cret.sel(factor_name=['SPY', 'IWM'])
px_line(da, title='SPY vs IWM', x='date', y='cret', color='factor_name')

In [ ]:
halflifes = HALFLIFES

draw_correlation(data.corr, factor_name=factor_name, factor_name_1=factor_name_1, corr_type=halflifes).show()
draw_volatility(data.vol, factor_name=factor_name, vol_type=halflifes).show()
draw_volatility(data.vol, factor_name=factor_name_1, vol_type=halflifes).show()

# Calculate week-over-week returns

In [ ]:
cret = data.cret.to_pandas()
# cret.sort_index(ascending=False).to_clipboard()

In [ ]:
factor_master = get_factor_master()

In [ ]:
factor_master['multiplier']

In [ ]:
ret_1d = calculate_returns_set(cret, periods=1, 
                      diffusion_map=factor_master['diffusion_type'], 
                      multiplier_map=factor_master['multiplier'])

ret_5d = calculate_returns_set(cret, periods=5, 
                      diffusion_map=factor_master['diffusion_type'], 
                      multiplier_map=factor_master['multiplier'])


In [ ]:
from risk_data import get_factor_master, get_factor_data
data = get_factor_data()
factor_master = get_factor_master() #data)
factor_master

In [ ]:
# from risk_stats import get_volatility_set_new
# from risk_config import HALFLIFES
# vol_1d = get_volatility_set_new(cret, halflifes=HALFLIFES)

# Betas

In [ ]:
COV_TYPES = {str(h): {'vol_type': h,
                      'corr_type': h}
             for h in HALFLIFES}
COV_TYPES

[(v['vol_type'], v['corr_type']) for k, v in COV_TYPES.items()]

In [ ]:
vol_0 = data.vol.sel(factor_name=factor_name).rename({'vol_type': 'cov_type'})
vol_1 = data.vol.sel(factor_name=factor_name_1).rename({'vol_type': 'cov_type'})
corr  = data.corr.sel(factor_name=factor_name, factor_name_1=factor_name_1).rename({'corr_type': 'cov_type'})
beta = (corr * vol_1 / vol_0).rename('beta')
beta

draw_beta(data, factor_name=factor_name, factor_name_1=factor_name_1).update_yaxes(type="log")

# Returns chart

In [ ]:
factor_data = get_factor_data()

In [ ]:
def draw_returns(ret, factor_1, factor_2):
    df = ret[[factor_1, factor_2]].reset_index()
    return px.bar(df, template='plotly_white', barmode='group')

In [ ]:
import plotly.express as px
factor_1 = 'QQQ'
factor_2 = 'IWM'

ret = factor_data.ret.to_pandas()[[factor_1, factor_2]]

df = ret.reset_index()
df
# px.bar(df, x='date', y=factor_1)
# px.bar(df, x='date', y=factor_2)

px.bar(ret[[factor_1, factor_2]].loc['2025':], template='plotly_white', barmode='group').update_traces(marker_line_width=0)
# px.bar(df, x='date', y=factor_1)

In [ ]:
df = factor_data.ret.to_pandas()[[factor_name, factor_name_1]] #.loc['2020':]
px.bar(df, template='plotly_white', barmode='group').show()

df2 = factor_data.ret.to_pandas()[[factor_name, factor_name_1]].reset_index() #.loc['2025':]
px.bar(df2, x='date', y=[factor_name, factor_name_1], template='plotly_white', barmode='group').update_traces(marker_line_width=0).show()


# Calcuate zscore

In [ ]:
data.vol.sel(vol_type=21).to_pandas()

In [ ]:
data.vol.sel(vol_type=21).shift({'date': 1}).to_pandas()

In [ ]:
data.ret.to_pandas().div(100)

In [ ]:
data.vol.sel(vol_type=63).to_pandas() / sqrt(252)

In [ ]:
zscore = data.ret / data.vol.shift({'date': 1})
zscore
# zscore.to_pandas()

# Calculate 200dMA

In [ ]:
cret_200dMA = data.cret.rolling({'date': 200}).mean()
cret  = data.cret

dist_ma = ((cret / cret_200dMA) - 1).rename('200dMA')

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
df = dist_ma.to_pandas()[['SPY', 'IWM']]
px.line(df, template='plotly_white')
# px.line(df)
# plt.show()

In [ ]:
from risk_chart import draw_distance_from_ma
factor_1 = 'SPY'
factor_2 = 'IWM'
draw_distance_from_ma(data.cret, factor_name=factor_1, factor_name_1=factor_2)

In [ ]:
from risk_data import get_factor_data
import risk_lib.stats as s
from importlib import reload
reload(s)

data = get_factor_data()
data['dist_ma'] = s.get_dist_ma_set(data.cret, [60, 200])
df = data.dist_ma.to_series().reset_index()
df


In [ ]:
from risk_chart import px_line
px_line(data.dist_ma.sel(factor_name=['SPY', 'IWM'], ma_type=200), 
        x='date', y='dist_ma', color='factor_name')

In [ ]:
window = 200
factor_name = 'SPY'
factor_name_1 = 'IWM'
_dist_ma = data.dist_ma.sel(factor_name=[factor_name, factor_name_1], ma_type=window)
fig = px_line(_dist_ma, x='date', y='dist_ma', color='factor_name', 
                title=f'Distance from {window}-day Moving Average (%)')
fig.show()